In [50]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from mp_api.client import MPRester

from pymatgen.core import Composition, Element
from pymatgen.analysis.phase_diagram import GrandPotentialPhaseDiagram, PhaseDiagram, PDPlotter
from pymatgen.analysis.interface_reactions import InterfacialReactivity, GrandPotentialInterfacialReactivity
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.compatibility import MaterialsProject2020Compatibility
from pymatgen.entries.computed_entries import ComputedEntry
from collections import defaultdict

import re
import sympy as sp




In [51]:
import contextlib
import io, sys
import siman  # program package to manage DFT calculations https://github.com/dimonaks/siman
from siman.calc_manage import smart_structure_read, get_structure_from_matproj
from siman.calc_manage import add, res
from siman import header
from siman.database import write_database, read_database
from siman.set_functions import read_vasp_sets
from siman.header import db
from siman.header import _update_configuration
from pydoc import importfile
from siman import thermo
from matplotlib import rc
import matplotlib.pyplot as plt

# Suppress output (replaces %%capture)
with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
    # Update configurations
    _update_configuration('./project_conf.py')
    read_database()  # read saved database if available
    
    project_sets = importfile('./project_sets.py')
    varset = read_vasp_sets(project_sets.user_vasp_sets, override_global=1)  # read user sets

    # Set paths
    # header.PATH2PROJECT = '../dft_calculations/'
    header.PATH2EDITOR = 'notepad.exe'
    header.PATH2POTENTIALS = "/home/a.burov/soft/vasp_potentials/potpaw_PBE_MPIE/"

    # Matplotlib settings (commented out as they're for interactive use)
    plt.rcParams['figure.dpi'] = 450

In [52]:
api_key = 'HbSHn48X1CSvBzRa4DOX5E9rhb5Tcc03'

In [4]:
def create_dataframe_from_calc(vasprun_path: str, calc_name: str, if_save: bool) -> None:
    """
        Inputs:
            vasprun_path - path to vasprun.xml
            calc_name - name to save a file
            if_save - save pandas dataframe as csv
        Returns:
            pandasDataframe - dataframe with the results
    """

    # Extraction of the calculation
    vasprun = Vasprun(vasprun_path)    
    # include structure so proper correction can be applied for oxides and sulfides
    reactant2 = vasprun.get_computed_entry(inc_structure=True)
    # reactant2.parameters['run_type'] = 'GGA+U'
    
    compatibility = MaterialsProject2020Compatibility()
    # compatibility.process_entry(reactant2)
    reactant2 = compatibility.process_entry(reactant2)

    reactant2.parameters['run_type'] = 'GGA_GGA+U'
    # reactant2.parameters['run_type'] = 'GGA+U'
    # reactant2.parameters['run_type'] = 'GGA+U'

    # Find coatings 
    comps = []
    emins = []
    reactions = []
    
    with MPRester(api_key=api_key) as mpr:
        for comp in tqdm(cathode_coatings.formula_pretty.unique()):
            try:
                # Is the system open to an elemental reservoir?
                grand = False
    
                if grand:
                    # Element in the elemental reservoir.
                    open_el = "Na" 
                    # Relative chemical potential vs. pure substance. Must be non-positive.
                    relative_mu = 3.25 # average intercalation potential
    
    
                # Get the compositions of the reactants
                reactant1 = comp
    
                comp1 = Composition(reactant1)
                comp2 = reactant2.composition # as it is manually calculated phase
    
                # Gather all elements involved in the chemical system.
                # elements = [e.symbol for e in comp1.elements + comp2.elements]
                elements = [e.symbol for e in comp1.elements + comp2.elements]
                
                if grand:
                    elements.append(open_el)
                elements = list(set(elements))  # Remove duplicates
    
                entries = mpr.get_entries_in_chemsys(elements, additional_criteria={'thermo_types': ['GGA_GGA+U', "R2SCAN"]})

                # Apply corrections locally with the mixing scheme
                scheme = MaterialsProjectDFTMixingScheme()
                corrected_entries = scheme.process_entries(entries)
    
                # Build a phase diagram using these entries.
                phase_diagram = PhaseDiagram(entries)
    
                # For an open system, include the grand potential phase diagram.
                if grand:
                    # Get the chemical potential of the pure subtance.
                    mu = phase_diagram.get_transition_chempots(Element(open_el))[0]
                    # Set the chemical potential in the elemental reservoir.
                    chempots = {open_el: relative_mu + mu}
                    # Build the grand potential phase diagram
                    gpd = GrandPotentialPhaseDiagram(entries, chempots)
                    # Create InterfacialReactivity object.
                    interface = GrandPotentialInterfacialReactivity(
                        comp1,
                        comp2,
                        gpd,
                        norm=True,
                        # include_no_mixing_energy=True,
                        include_no_mixing_energy=False,
                        pd_non_grand=phase_diagram,
                        use_hull_energy=True,
                    )
                    
                else:
                    interface = InterfacialReactivity(
                        comp1,
                        comp2,
                        phase_diagram,
                        norm=True,
                        include_no_mixing_energy=False,
                        # include_no_mixing_energy=False,
                        pd_non_grand=None,
                        use_hull_energy=True,
                    )
                    
                interface_table = interface.get_dataframe()
                emins.append(interface.get_dataframe()['E$_{\textrm{rxn}}$ (eV/atom)'].min())
                comps.append(comp)
                index = np.argmin(interface_table['E$_{\textrm{rxn}}$ (eV/atom)'].values)
                reactions.append(interface_table['Reaction'].iloc[index])
            except:
                continue

    res = pd.DataFrame()
    res['formula_pretty'] = comps
    res[f'e_rxn_{calc_name}_CC'] = emins
    res[f'interface_reaction_{calc_name}'] = reactions

    if if_save:
        res.merge(cathode_coatings).round(3).to_csv(f'data_nmc/{calc_name}_CC_calculation_results.csv', index = False)
        
    return res
    
    
    


In [5]:
# Path to vasprun.xml file
vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_inter.xml"



In [ ]:
# Parse vasprun.xml
vasprun = Vasprun(vasprun_path)

# Get final energy
final_energy = vasprun.final_energy
print(f"Final energy: {final_energy} eV")


In [ ]:
vasprun.final_structure.num_sites

In [ ]:
# Access the structure
structure = vasprun.final_structure
# print(structure)


## Thermodynamic stability

In [6]:
from pymatgen.entries.compatibility import MaterialsProjectCompatibility


In [7]:
def parse_formula(formula):
    pattern = r'([A-Z][a-z]?)(\d*)|(\()|(\))(\d*)'
    matches = list(re.finditer(pattern, formula))
    
    stack = [defaultdict(int)]
    
    for match in matches:
        element, count, open_paren, close_paren, paren_count = match.groups()
        
        if element:
            cnt = int(count) if count else 1
            stack[-1][element] += cnt
        
        elif open_paren:
            stack.append(defaultdict(int))
        
        elif close_paren:
            cnt = int(paren_count) if paren_count else 1
            group = stack.pop()
            for elem, num in group.items():
                stack[-1][elem] += num * cnt
    
    return dict(stack.pop())




In [152]:
# Path to vasprun.xml file
# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_inter.xml"
# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_deinter.xml"
# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_some_na.xml"

# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_al.xml"
# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_al_some_na.xml"
# vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_al_some_na_dead.xml"

vasprun_path = "/home/arseniy/Desktop/work/nvpf_al_doped/data/stability/vasprun_al_some_na_dead.xml"



In [153]:
# Parse vasprun.xml
vasprun = Vasprun(vasprun_path)

# Get final energy
final_energy = vasprun.final_energy
print(f"Final energy: {final_energy} eV")


Final energy: -851.87798644 eV eV


In [154]:
comp_entry = ComputedEntry(vasprun.final_structure.composition, vasprun.final_energy)


In [155]:
elements = [el.symbol for el in comp_entry.elements]  # elements in your compound

with MPRester(api_key) as mpr:
    entries = mpr.get_entries_in_chemsys(elements)  # all known phases in this system

    

Retrieving ThermoDoc documents:   0%|          | 0/1365 [00:00<?, ?it/s]

In [156]:
compat = MaterialsProjectCompatibility()
# MaterialsProject2020Compatibility
entries = compat.process_entries(entries)

# Also process your entry with the same corrections
# entry = compat.process_entry(comp_entry)


/tmp/ipykernel_316367/992659018.py:1: DeprecationWarning: MaterialsProjectCompatibility is deprecated; use MaterialsProject2020Compatibility instead.
Materials Project formation energies use the newer MaterialsProject2020Compatibility scheme.
  compat = MaterialsProjectCompatibility()


In [157]:
def balance_reaction_one_reactant_any_products(reactants, products):
    # Get the single reactant name and atom counts
    reactant_name = next(iter(reactants))
    reactant_counts = reactants[reactant_name]

    # Collect unique elements from reactant and all products
    elements = sorted(set(reactant_counts.keys()).union(
        el for p in products.values() for el in p.keys()
    ))

    # Create sympy variables: one for reactant 'a' and one for each product
    a = sp.symbols('a')
    product_vars = sp.symbols(f'b0:{len(products)}')

    # Build balance equations for each element
    equations = []
    for el in elements:
        lhs = a * reactant_counts.get(el, 0)  # reactant side
        rhs = sum(product_vars[i] * p.get(el, 0) for i, p in enumerate(products.values()))
        equations.append(sp.Eq(lhs, rhs))

    # Solve for product coefficients in terms of 'a'
    vars_to_solve = list(product_vars)
    solutions = sp.solve(equations, vars_to_solve, dict=True)
    if not solutions:
        raise ValueError("No solution found to balance the equation.")
    sol = solutions[0]
    sol[a] = a  # keep 'a' free

    # Substitute a=1 to get numeric coefficients
    coeffs = {a: 1}
    coeffs.update({var: sol[var].subs(a, 1) for var in product_vars})

    # Normalize coefficients to smallest integers
    def to_int_fraction(x):
        if isinstance(x, int):
            return x, 1
        else:
            p, q = x.as_numer_denom()
            return int(p), int(q)

    nums_dens = [to_int_fraction(v) for v in coeffs.values()]
    from math import gcd
    from functools import reduce

    def lcm(x, y):
        return x * y // gcd(x, y)

    common_den = reduce(lcm, (d for _, d in nums_dens), 1)
    scaled = [n * (common_den // d) for n, d in nums_dens]
    gcd_all = reduce(gcd, scaled)
    normalized = [s // gcd_all for s in scaled]

    # Prepare output dictionary with species names and their coefficients
    balanced_coeffs = {reactant_name: normalized[0]}
    for i, pname in enumerate(products):
        balanced_coeffs[pname] = normalized[i + 1]

    return balanced_coeffs



In [158]:
st_names = {
        "mp-1238774": 'navpo4f',
        # "mp-25440": 'vpo4f',     
        "mp-686540": 'na3al2p2o8f3',     
        "mp-7848": 'alpo4',     
        "mp-19291": 'navpo5',     
        "mp-468": 'alf3',     
        "mp-559931": 'vf3',     
        "mp-1094113": 'NaV2P2O9',     
        "mp-1104878": 'VPO4F',     
        "mp-17999": 'NaVF4',     
        "mp-752810": 'NaVP2O7',     
        "mp-694937": 'Na3V2P2O8F3',     
        "mp-18835": 'VPO4',     
        "mp-25265": 'VPO5',     
        "mp-26728": 'VP2O7',     
        "mp-1102077": 'NaAlF4',     
       }


In [159]:
all_entries = entries + [comp_entry]
pd = PhaseDiagram(all_entries)

decomp, e_above_hull = pd.get_decomp_and_e_above_hull(comp_entry)

reactants = {vasprun.final_structure.reduced_formula: parse_formula(vasprun.final_structure.reduced_formula)}
reactant_name = next(iter(reactants))
reactant_at = sum(reactants[reactant_name].values())

products = {}

print(f"Energy above hull: {e_above_hull:.4f} eV/atom")
if e_above_hull == 0:
    print("Phase is thermodynamically stable")
elif e_above_hull < 0.05:
    print("Phase is metastable")
else:
    print("Phase decomposes to:")
    for entry, amount in decomp.items():
        mp_id = getattr(entry, "entry_id", "N/A")
        products[entry.composition.reduced_formula] = parse_formula(entry.composition.reduced_formula)
        print(f"{amount:.3f}*{entry.composition.reduced_formula} + ", end="")

    bal = balance_reaction_one_reactant_any_products(reactants, products)
    
    print()
    coeff_react = bal[reactant_name]
    en_decomp = (vasprun.final_energy / vasprun.final_structure.num_sites) * reactant_at * coeff_react
    for entry, amount in decomp.items():
        mp_id = getattr(entry, "entry_id", "N/A")
        mp_id_parse = mp_id.split('-GGA')[0]
        react_name = entry.composition.reduced_formula
        coeff_prod = bal[react_name]
        name_calc = st_names[mp_id_parse]
        en_react = db[name_calc, '9_bulk_mp', 1].e0_at
        product_at = sum(products[entry.composition.reduced_formula].values())
        en_decomp -= en_react * product_at * coeff_prod

    en_decomp /= (reactant_at * coeff_react)
    # en_decomp /= (reactant_at)
    
    print(f"Decomposition energy is {en_decomp*1000:.0f} meV/atom")

        # print(f"  {entry.composition.reduced_formula} (MP-ID: {mp_id}) : {amount:.3f}")
        

Energy above hull: 0.3223 eV/atom
Phase decomposes to:
0.241*NaV2P2O9 + 0.483*VPO4F + 0.172*VP2O7 + 0.103*NaAlF4 + 
Decomposition energy is 3 meV/atom


In [160]:
coeff_react

1

In [161]:
bal


{'Na2AlV7P8(O4F)8': 1, 'NaV2P2O9': 1, 'VPO4F': 4, 'VP2O7': 1, 'NaAlF4': 1}

In [57]:
with MPRester(api_key) as mpr:
    docs = mpr.materials.summary.search(
        material_ids=["mp-149",]
    )

    

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [63]:
print(docs[0].dict())  # Prints all data fields for this material



{'builder_meta': {'emmet_version': '0.84.3rc4', 'pymatgen_version': '2024.11.13', 'run_id': '4e80c83d-fc3d-4e2d-8bf5-affe2b0daa98', 'batch_id': None, 'database_version': '2025.06.09', 'build_date': '2024-11-21 22:30:12.439000', 'license': 'BY-C'}, 'nsites': 2, 'elements': ['Si'], 'nelements': 1, 'composition': {'Si': 2.0}, 'composition_reduced': {'Si': 1.0}, 'formula_pretty': 'Si', 'formula_anonymous': 'A', 'chemsys': 'Si', 'volume': 40.32952684741405, 'density': 2.312800253345134, 'density_atomic': 20.164763423707026, 'symmetry': {'crystal_system': 'Cubic', 'symbol': 'Fd-3m', 'number': 227, 'point_group': 'm-3m', 'symprec': 0.1, 'angle_tolerance': 5.0, 'version': '2.5.0'}, 'property_name': 'summary', 'material_id': 'mp-149', 'deprecated': False, 'deprecation_reasons': None, 'last_updated': '2025-06-03 23:06:51.992000', 'origins': [{'name': 'structure', 'task_id': 'mp-1947498', 'last_updated': '2021-03-05 11:47:03.084000'}, {'name': 'energy', 'task_id': 'mp-1947498', 'last_updated': '2

In [64]:
task_ids = mpr.get_task_ids_associated_with_material_id("mp-149")
print(task_ids)

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

['mp-655585', 'mp-656511', 'mp-655936', 'mp-11721', 'mp-149', 'mp-1057373', 'mp-1057366', 'mp-1057380', 'mp-1059585', 'mp-1059589', 'mp-1059603', 'mp-1120258', 'mp-1120259', 'mp-1141021', 'mp-1248038', 'mp-1249516', 'mp-1267607', 'mp-1440634', 'mp-1686587', 'mp-1791788', 'mp-1594776', 'mp-1592727', 'mp-1947498', 'mp-1950734', 'mp-1059604', 'mp-1057384', 'mp-1536661', 'mp-2064724', 'mp-2064214', 'mp-2250750', 'mp-2299819', 'mp-2291052', 'mp-2693536', 'mp-2693792', 'mp-2683378', 'mp-2768327', 'mp-2375705', 'mp-2375783', 'mp-2375896', 'mp-2629333', 'mp-2629357', 'mp-2683297']


In [83]:
with MPRester(api_key) as mpr:
    # Search for tasks by their task_id field
    results = mpr.tasks.search(task_ids='mp-1095269')

    for task in results:
        print(f"Task ID: {task.task_id}")
        print(f"Calculation Type: {task.task_type }")
        # print(f"POTCAR Symbols: {task.potcar_symbols}")
        # print(f"INCAR Parameters: {task.input.incar_parameters}")
        print("---")

        

/tmp/ipykernel_4716/28797834.py:3: DeprecationWarning: Accessing tasks data through MPRester.tasks is deprecated. Please use MPRester.materials.tasks instead.
  results = mpr.tasks.search(task_ids='mp-1095269')


Retrieving TaskDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Task ID: mp-1095269
Calculation Type: Structure Optimization
---


In [84]:
results[0].calcs_reversed


[Calculation(dir_name='/clusterfs/mp/mp_prod/block_2018-04-27-09-04-20-960461/launcher_2018-04-27-13-43-35-068128', vasp_version='5.4.4', has_vasp_completed=True, input=CalculationInput(incar={'ISTART': 1, 'PREC': 'accurate', 'ALGO': 'Fast', 'ISPIN': 2, 'ICHARG': 1, 'NELM': 100, 'IBRION': 2, 'EDIFF': 0.0006000000000000001, 'NSW': 99, 'ISIF': 3, 'ENCUT': 520.0, 'MAGMOM': [0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001, 0.6000000000000001], 'LREAL': 'Auto', 'ISMEAR': -5, 'SIGMA': 0.05, 'LWAVE': False, 'LORBIT': 11, 'KPOINT_BSE': [-1, 0, 0, 0]}, kpoints=Kpoints from vasprun.xml
 0
 Gamma
 7 4 2
 , nkpoints=None, potcar=['Si'], potcar_spec=[PotcarSpec(titel='PAW_PBE Si 05Jan2001', hash='b2b0ea6feb62e7cde209616683b8f7f5', summary_stats=None)], potcar_type=['PAW_PBE'], parameters={'SYSTEM': 'unknown system', 'LCOMPAT': Fal